In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Germany'
chdir(csv_file_path)

Y = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)
X = X_all['2002-05-01':' 2016-12-31']
X

,rr,tg,tn,tx,pp,hu,fg,qq,et
time,,,,,,,,,
2002-05-01,0.0,12.990000,7.42,17.630000,1008.9000,70.520004,2.88,205.0,2.765489
2002-05-02,0.0,11.759999,7.08,16.109999,1008.0000,79.690000,2.70,154.0,2.015525
2002-05-03,0.0,11.880000,8.66,16.779999,1007.7000,89.940000,2.49,102.0,1.339214
2002-05-04,6.3,8.670000,5.98,10.950000,1009.5000,94.128580,2.40,85.0,1.022122
2002-05-05,7.0,7.870000,3.90,13.219999,1015.2000,77.665000,1.87,137.0,1.604829
...,...,...,...,...,...,...,...,...,...
2016-12-27,0.0,2.470000,1.15,3.530000,1039.9000,84.810000,4.71,27.0,0.260880
2016-12-28,0.0,3.510000,0.50,4.260000,1043.1000,85.750000,3.62,28.0,0.280443
2016-12-29,0.0,-1.310000,-4.30,4.440000,1042.1000,95.075005,2.09,51.0,0.424098


In [2]:
X_train = X[:'2011-12-31']
X_test = X['2014-01-01':]
X_valid = X['2012-01-01':'2013-12-31']
Y_train = Y[:'2011-12-31']
Y_test = Y['2014-01-01':]
Y_valid = Y['2012-01-01':'2013-12-31']

csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *
# data is normalized per column
(dataset_train,dataset_test,dataset_valid,scaler_X,scaler_Y)=prepare_data(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)
 
iters=3
n_steps_in=30
n_steps_out=1

# windows of 30 time steps as input and 1 time step (daily) as output are created
x, y = split_sequences_y1(dataset_train, n_steps_in, n_steps_out)
x_test, y_test = split_sequences_y1(dataset_test, n_steps_in, n_steps_out)
x_valid, y_valid = split_sequences_y1(dataset_valid, n_steps_in, n_steps_out)

In [3]:
# this block is to set dropout active during training and testing (for Monte Carlo Dropout)
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [4]:
# Transformer model is mostly in helper function
from keras_tuner import HyperModel
from tensorflow.keras import layers

class Transformer_HyperModel(HyperModel):
    def __init__(self, input_shape, output_shape):
        super(Transformer_HyperModel, self).__init__()
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        
    def build(self, hp):# 3 tunable parameters
        head_size = hp.Int("head_size", min_value=8, max_value=64) #, default=16
        num_heads = 1
        ff_dim = hp.Int("ff_dim", min_value=1, max_value=8) #, default=4
        num_transformer_blocks = 2 #, default=4
        mlp_units = hp.Int("mlp_units", min_value=10, max_value=200)
        dropout = 0.1
        mlp_dropout = 0.1
        
        model = build_model(self.input_shape, self.output_shape, head_size,num_heads,ff_dim,
                                num_transformer_blocks,mlp_units,mlp_dropout)
        model.compile(loss="mean_absolute_error",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),metrics=["mean_absolute_error"])
        return model

input_shape = x.shape[1:]
output_shape = y.shape[1]

hyper_model = Transformer_HyperModel(input_shape, output_shape)

es_callback = keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error',restore_best_weights=True, patience=4)

In [5]:
# tuning the models using random search
tuner= kt.RandomSearch(
        hyper_model,
        overwrite=True,
        objective='val_mean_absolute_error',
        max_trials = 10,
        directory='D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge',
        project_name='TF Germany'
        )

tuner.search(
        x,
        y,
        batch_size=24,
        epochs=30,
        validation_data=(x_valid,y_valid),
        callbacks=[es_callback],
        verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]

Y_pred=best_model.predict(x_test)
best_model.summary()

Trial 10 Complete [00h 03m 05s]
val_mean_absolute_error: 0.08501734584569931

Best val_mean_absolute_error So Far: 0.060606248676776886
Total elapsed time: 00h 18m 02s
INFO:tensorflow:Oracle triggered exit


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        906         layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 30, 9)        0           multi_head_attention[0][0]   

In [6]:
tuner.results_summary()

#saving the model
csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)
from tensorflow.keras.models import save_model
best_model.save('TF_Germany_tune.h5')

NSE=np.array([[None]*n_steps_out])
MAE=np.array([[None]*n_steps_out])
RMSE=np.array([[None]*n_steps_out])
SAPE=np.array([[None]*n_steps_out])

Y_pred_test=best_model.predict(x_test)
y_correct_utf = scaler_Y.inverse_transform(y_test) # test
yhat_utf = scaler_Y.inverse_transform(Y_pred_test) # test

NSE[:]= np.array([1 - sum((yhat_utf[:,i]-y_correct_utf[:,i])**2)/sum((y_correct_utf[:,i]-np.mean(y_correct_utf[:,i]))**2)
                  for i in range(n_steps_out)])
MAE[:]=np.array([mean_absolute_error(y_correct_utf[:,i], yhat_utf[:,i]) for i in range(n_steps_out)])
RMSE[:]=np.array([np.sqrt(np.mean((yhat_utf[:,i]-y_correct_utf[:,i])**2)) for i in range(n_steps_out)])
SAPE[:] = np.array([np.abs((yhat_utf[:,i] - y_correct_utf[:,i])/(0.5*(y_correct_utf[:,i]+yhat_utf[:,i]))).mean()
                         for i in range(n_steps_out)]) #mean absolute percent error
print(np.mean(MAE))

Results summary
Results in D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\TF Germany
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
head_size: 23
ff_dim: 5
mlp_units: 128
Score: 0.060606248676776886
Trial summary
Hyperparameters:
head_size: 28
ff_dim: 1
mlp_units: 142
Score: 0.06879517436027527
Trial summary
Hyperparameters:
head_size: 39
ff_dim: 5
mlp_units: 160
Score: 0.06976642459630966
Trial summary
Hyperparameters:
head_size: 49
ff_dim: 5
mlp_units: 21
Score: 0.07517752796411514
Trial summary
Hyperparameters:
head_size: 18
ff_dim: 1
mlp_units: 112
Score: 0.07579997926950455
Trial summary
Hyperparameters:
head_size: 20
ff_dim: 8
mlp_units: 197
Score: 0.0847395807504654
Trial summary
Hyperparameters:
head_size: 48
ff_dim: 2
mlp_units: 70
Score: 0.08501734584569931
Trial summary
Hyperparameters:
head_size: 27
ff_dim: 4
mlp_units: 185
Score: 0.08571317046880722
Trial summary
Hyperparameters:
head_size: 52
ff_d